In [11]:
dataset_name = ["nfcorpus", "trec-covid", "climate-fever"][2]
num_positives = 16
num_negatives_per_positive = 4

In [12]:
# Load entire corpus
import json

def load_jsonl(path):
    with open(path) as f:
        return [json.loads(i) for i in f.readlines()]
        
corpus = load_jsonl(f"/gallery_louvre/dayoon.ko/research/sds/src/datasets/{dataset_name}/corpus.jsonl")
corpus_dict = {}
for item in corpus:
    corpus_dict[item["_id"]] = item["text"]

In [88]:
model_name = ["bge-large-en-v1.5", "contriever", "gte-base", "multilingual-e5-large"][2]
dropout=["",  "-0.02", "-0.05"][0]

In [89]:
model_name, dataset_name, dropout

('gte-base', 'climate-fever', '')

In [90]:
# Load succ and fail csv files
import pandas as pd
path = f"{model_name}/{dataset_name}-n-query-mt-2.csv"
df = pd.read_csv(path)
df = df.drop('Unnamed: 0', axis=1)
df = df.set_index("corpus-id")
df
# Get succ and fail corpus-id
#meta = df.to_dict()
#print(meta)

,n-query,recall,precision,f1
corpus-id,,,,
Second_law_of_thermodynamics,4,1.000000,1.000000,1.000000
Polar_bear,11,1.000000,0.785714,0.880000
Jet_stream,7,1.000000,0.777778,0.875000
Pluto,3,1.000000,0.750000,0.857143
Conservative_Party_(UK),3,0.666667,1.000000,0.800000
...,...,...,...,...
Republican_Party_(United_States),3,0.000000,0.000000,0.000000
Gaia_hypothesis,3,0.000000,0.000000,0.000000
Botany,4,0.000000,0.000000,0.000000


In [91]:
def filter_data(res):
    for i in res:
        i["retrieval"] = [j for j in i["retrieval"] if i["_id"] != j]
    return res

In [94]:
def get_ret_map(pth):
    pth0 = path.replace(".csv", f"-d2d-retrieval.jsonl")
    res0 = load_jsonl(pth0)
    res0 = filter_data(res0)
    res0 = {i["_id"]: i["retrieval"] for i in res0}
    pth1 = path.replace(".csv", f"-d2d-retrieval{dropout}.jsonl")
    res1 = load_jsonl(pth1)
    res1 = filter_data(res1)
    res1 = {i["_id"]: i["retrieval"] for i in res1}
    pth2 = path.replace(".csv", f"-d2d2d-retrieval{dropout}.jsonl")
    res2 = load_jsonl(pth2)
    res2 = filter_data(res2)
    res2 = {i["_id"]: i["retrieval"][1:] for i in res2}
    results = {}
    for qid, ret_ids in res1.items():
        selected_rets = {}
        for ret_id in ret_ids[:num_positives]:
            try:
                ret2_ids = res2[ret_id]
            except:
                print(ret_id)
                continue
            ret2_ids_selected = [i for i in ret2_ids if i not in res0[qid] and i != qid]
            ret2_ids_selected = ret2_ids_selected[:num_negatives_per_positive]
            if len(ret2_ids_selected) < num_negatives_per_positive:
                continue
            selected_rets[ret_id] = ret2_ids_selected
            if len(selected_rets) == num_positives:
                break
        results[qid] = selected_rets
    return results
        
ret_map = get_ret_map(path)

Nicolas_Léonard_Sadi_Carnot
Thermodynamics
Reflections_on_the_Motive_Power_of_Fire
Entropy
Carnot's_theorem_(thermodynamics)
Theory_of_heat
Carnot_heat_engine
Rudolf_Clausius
Carnot_cycle
History_of_entropy
Fluctuation_theorem
Philosophy_of_thermal_and_statistical_physics
Entropy_(classical_thermodynamics)
H-theorem
Polar_Bear_(disambiguation)
Ursus_maritimus_tyrannus
Polar_seas
Brown_bear
Kodiak_bear
Carnivora
Walrus
Marine_mammal
Polar_Bears_International
Alaska_Peninsula_brown_bear
Polar_bear_hunting
Bergman's_bear
Mountain_jet
Thermal_wind
Polar_vortex
Airstream_Jets
British_Aerospace_Jetstream
Weather
Tropical_Easterly_Jet
Atmospheric_river
Clear-air_turbulence
Rear-inflow_jet
Charon_(moon)
Kuiper_belt
Eris_(dwarf_planet)
Plutoid
Dwarf_planet
Pluto_(disambiguation)
Moons_of_Pluto
List_of_the_brightest_Kuiper_belt_objects
Pluto_Kuiper_Express
2015_RR245
Trans-Neptunian_objects_in_fiction
Ceres_(dwarf_planet)
Neptune
History_of_the_Conservative_Party_(UK)
Scottish_Conservatives
Cons

In [93]:
def get_dataset(ret_map):
    dataset = []
    for query_id, rets1 in ret_map.items():
        for pos_id, neg_ids in rets1.items():
            query_meta = df.loc[query_id]
            query = corpus_dict[query_id]
            positive = [corpus_dict[pos_id]]
            negatives = [corpus_dict[idx] for idx in neg_ids]
            dataset.append({
                "_id": query_id,
                "recall": query_meta["recall"],
                #"precision": query_meta["precision"],
                #"f1": query_meta["f1"],
                "query": query,
                "pos": positive,
                "neg": negatives,
            })
    return dataset 

dataset = get_dataset(ret_map)
            

In [87]:
print(f"/gallery_louvre/dayoon.ko/research/sds/eval_retrieval/gradnorm/{model_name}/{dataset_name}{dropout}.json")
with open(f"/gallery_louvre/dayoon.ko/research/sds/eval_retrieval/gradnorm/{model_name}/{dataset_name}{dropout}.json", "w") as f:
    json.dump(dataset, f, indent=2)

/gallery_louvre/dayoon.ko/research/sds/eval_retrieval/gradnorm/contriever/climate-fever.json


#### Deprecated

In [20]:
import random

negs = [i["text"] for i in random.sample(corpus, 20) if len(i["text"]) > 0][:5]

def generate_rand_neg_dataset(docs, positives, negatives):
    dataset = []
    for i in range(len(docs)):
        query = docs[i]
        positives_selected = []
        negatives_selected = []
        for j, doc in enumerate(positives[i]):
            if len(negatives[i][j]) == num_negatives_per_positive:
                positives_selected.append(doc)
                negatives_selected.append(negs)
            if len(positives_selected) == 4:
                break
        for p, ns in zip(positives_selected, negatives_selected):
            dataset.append({
                "query": query,
                "pos": [p],
                "neg": ns
            })
    return dataset

In [21]:
succ_dataset = generate_rand_neg_dataset(succ_docs, succ_positives, succ_negatives)
print(succ_dataset[0])
with open("id_randneg_dataset/trec-covid-succ.json", "w") as f:
    json.dump(succ_dataset, f, indent=2)


fail_dataset = generate_rand_neg_dataset(fail_docs, fail_positives, fail_negatives)
print(fail_dataset[0])
with open("ood_randneg_dataset/trec-covid-fail.json", "w") as f:
    json.dump(fail_dataset, f, indent=2)

NameError: name 'succ_docs' is not defined

In [ ]:
succ_negatives = get_negatives(succ_path)
succ_negatives_selected = [succ_negatives[i] for i in succ_ids_selected]

In [ ]:
def generate_dataset(docs, positives, negatives):
    dataset = []
    for i in range(len(docs)):
        query = docs[i]
        positives_selected = []
        negatives_selected = []
        for j, doc in enumerate(positives[i]):
            if len(negatives[i][j]) == num_negatives_per_positive:
                positives_selected.append(doc)
                negatives_selected.append(negatives[i][j])
            if len(positives_selected) == 4:
                break
        for p, ns in zip(positives_selected, negatives_selected):
            dataset.append({
                "query": query,
                "pos": [p],
                "neg": ns
            })
    return dataset


In [ ]:
succ_dataset_selected = generate_dataset(succ_docs_selected, succ_positives_selected, succ_negatives_selected)
print(succ_dataset_selected[0])
with open("trec-covid/ood_well_over_0.4.json", "w") as f:
    json.dump(succ_dataset_selected, f, indent=2)

## OOD all consecutive

In [ ]:
pth = "/gallery_louvre/dayoon.ko/research/sds/FlagEmbedding/FlagEmbedding/BGE_M3/"

def load_file(pth):
    with open(pth) as f:
        lines = f.readlines() 
    return lines 
    
def get_recall_and_score_list(pth):
    lines = load_file(pth)
    gradnorms = []
    for l in lines:
        try: 
            sid = l.index("tensor(") + len("tensor(") 
            eid = sid + 5
            gradnorms.append(float(l[sid:eid]))
        except:
            continue
    recalls = []
    for l in lines:
        try: 
            sid = l.index("recall: ") + len("recall: ") 
            eid = sid + 5
            recalls.append(int(float(l[sid:eid].strip()) * 100))
        except:
            continue
    ids = []
    for l in lines:
        try:
            sid = l.index("doc_id: ") + len("doc_id: ") 
            eid = sid
            ids.append(l[sid:].strip())
        except:
            continue

    gradnorms_selected = []
    recalls_selected = []
    ids_selected = []
    func = min #lambda x: sum(x) / len(x)
    for i in range(0, len(ids), 4):
        gradnorms_selected.append(func(gradnorms[i:i+4]))
        recalls_selected.append(func(recalls[i:i+4]))
        ids_selected.append(ids[i])
            
    return recalls_selected, gradnorms_selected, ids_selected



ood_all = pth + "ood_all"
recalls, gradnorms, ids = get_recall_and_score_list(ood_all)
print(len(recalls), len(gradnorms), len(ids))

import matplotlib.pyplot as plt 
plt.figure()
plt.scatter(recalls, gradnorms, color="green")
plt.xlabel("recall")
plt.ylabel("gradient norm")
#plt.plot(ood_well_o, len(ood_well_o) * [0], "o" , label = "ood-well(tp_over_t>0.4)", color="blue")
#plt.plot(ood_poor, len(ood_poor) * [1], "x" , label = "ood-poor", color="red")

means = {k: [] for k in set(recalls)}
for r, g in zip(recalls, gradnorms):
    means[r].append(g)
means = {k: sum(v) / len(v) for k, v in means.items()}
means = sorted(list(means.items()), key= lambda x: int(x[0]))
xs = [i[0] for i in means]
ys = [i[1] for i in means]
plt.scatter(xs, ys, marker="x", color="red")
plt.legend()
 

In [ ]:
data = {
    "_id": ids,
    "recall": recalls,
    "grad_norms": gradnorms,
    "scores": scores
}
import pandas as pd
df = pd.DataFrame(data)
df = df.sort_values("recall")

In [ ]:
df.head(20)

In [ ]:
df.tail(20)